In [ ]:
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:90% !important; }</style>"))

In [1]:
import os, sys
import argparse
import os.path as osp
import random
from time import perf_counter
import yaml
import numpy as np
import scipy.sparse as sp
import scipy

import torch
import torch.nn.functional as F
import torch.nn as nn

import logging
import shutil
import ast

from lib_EGNN_Pytorch.model import RwCL_Model
from lib_EGNN_Pytorch import utils, Post_utils, evaluation
from lib_EGNN_Pytorch.data_preprocessing import Pre_utils

from lib_EGNN_Pytorch.app.multi_exec import execute_train_investigate, execute_test_investigate
%matplotlib inline  

In [2]:
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

# Set up logging
logger = logging.getLogger()
logger.handlers = []
ch = logging.StreamHandler()
formatter = logging.Formatter(
        fmt='%(asctime)s (%(levelname)s): %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S')
ch.setFormatter(formatter)
logger.addHandler(ch)
logger.setLevel('INFO')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
# >>>>>>>>>>>>>>>>>>>> Setting input data and configurations:
tkipf_graph_path = "/home/xiangli/projects/tmpdata/GCN/Graph_Clustering/tkipf_gcn_data"

data_name = 'cora'
# data_name = 'pubmed'
# data_name = 'cite'

# data_name = 'acm'
# data_name = 'dblp'
# data_name = 'cite'

sdcn_data_path = "/home/xiangli/projects/tmpdata/GCN/Graph_Clustering/sdcn/"
workdir = f"/home/xiangli/projects/GCN_program/Workshop_local/EGNN_workdir_results/{data_name}/"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

config_file_name = f'config_{data_name.lower()}.yaml'
config_file_path = os.path.join('./config_data_RwCL_clustering/', config_file_name)
with open(config_file_path, 'r') as c:
    config = yaml.safe_load(c)    
    
# For strings that yaml doesn't parse (e.g. None)
for key, val in config.items():
    if type(val) is str:
        try:
            config[key] = ast.literal_eval(val)
        except (ValueError, SyntaxError):
            pass

torch.manual_seed(config['seed'])
random.seed(12345)        
        
print(">>>>>>>>>>>>>>>>>>>> Setting Tuning :")
tune_param_name = "nothing"
# tune_param_name = "tsne_1"
# tune_param_name = "dropout_rate"
# tune_param_name = "n_epochs"
# tune_param_name = "weight_decay"

tune_val_label_list = [1]
# tune_val_label_list = [0.0, 0.1, 0.2, 0.5]

# tune_val_list = [10**(-val) for val in tune_val_label_list]
tune_val_list = [val for val in tune_val_label_list]

# trainer_id_list = [0]
trainer_id_list = list(range(1))

print(f"Tune param : {tune_param_name} ; with the following values: {tune_val_list}")

print(">>>>>>>>>>>>>>>>>>>> Loading configs :")
for key, val in config.items():
    print(f"{key}    :    {val}")
    
# =================== copy the config file: =======================================
dest_folder = os.path.dirname(os.path.join(workdir, f"tune_{tune_param_name}/"))
if not os.path.exists(os.path.join(dest_folder, config_file_name)):    
    os.makedirs(dest_folder, exist_ok=True)
    shutil.copyfile(config_file_path,  os.path.join(dest_folder, config_file_name))    

>>>>>>>>>>>>>>>>>>>> Setting Tuning :
Tune param : nothing ; with the following values: [1]
>>>>>>>>>>>>>>>>>>>> Loading configs :
data_name    :    cora
n_clusters    :    7
feat_dim    :    1433
seed    :    39788
lr    :    0.0001
enc_arch    :    256-128
mlp_arch    :    256-
num_proj_hidden    :    512
tau    :    1.0
drop_feature_rate_1    :    0.08
view_num    :    3
num_epochs    :    300
weight_decay    :    0.02
batch_size_train    :    512
eval_display    :    10
loss_batch_size    :    0
alpha    :    0.1
rmax    :    1e-06
rrz    :    0.4
batchnorm    :    False
dropout_rate    :    0.1


### Use tkipf dataset: Cora, PubMed

In [4]:
Cython_GBP_data_path = f"/home/xiangli/projects/tmpdata/GCN/Graph_Clustering/tkipf_gcn_data/Packed_data/no_row_normalize/{data_name.lower()}/GBP_input/"
# Pre_utils.convert_GBP_input_tkipf_gcn(data_name.lower(), tkipf_graph_path, directed = False,  
#                                       normalize = False, redo_save = False)

adj_full, features, labels_full, _ = Pre_utils.load_gcn_tkipf_data(tkipf_graph_path, 
                                                        data_name.lower(), normalize = False, redo_save = False)

features = np.ascontiguousarray(features, dtype = np.float32)
adj_matrix_cython = np.ascontiguousarray(np.load(os.path.join(Cython_GBP_data_path, f'{data_name.lower()}_adj.npy')), dtype=np.int64)



Packed data already exists at: /home/xiangli/projects/tmpdata/GCN/Graph_Clustering/tkipf_gcn_data/Packed_data/no_row_normalize/cora, LOADING...


### SDCN dataset: Cite, ACM, DBLP

In [ ]:
Cython_GBP_data_path = os.path.join(sdcn_data_path, f"data_for_GBP_input/{data_name.lower()}/") 
Pre_utils.convert_GBP_input_sdcn(data_name, sdcn_data_path, target_path = Cython_GBP_data_path)

features, labels_full = Pre_utils.load_sdcn_data_func(sdcn_data_path, data_name)  # both features and labels are numpy array
adj_full = Pre_utils.load_sdcn_graph(sdcn_data_path, data_name)  # scipy.sparse.csr_matrix


features = np.ascontiguousarray(features, dtype = np.float32)
adj_matrix_cython = np.ascontiguousarray(np.load(os.path.join(Cython_GBP_data_path, f'{data_name.lower()}_adj.npy')), dtype=np.int64)

features_GBP = Pre_utils.GBP_feat_precomputation(data_name, 40, 
                                config["alpha"], config["rmax"], config["rrz"], 
                                rwnum = 0, directed = False, add_self_loop = True,
                                rand_seed = 10, 
                                feats = features, adj_matrix = adj_matrix_cython)

emb_stat_obj = evaluation.semantic_emb(labels_full)


print(sum(adj_full.diagonal()))

### Execute Tuning

In [7]:
for tune_val_label, tune_val in zip(tune_val_label_list, tune_val_list):
    
    features_GBP = Pre_utils.precompute_Cython_GBP_feat(data_name, 40, 
                                config["alpha"], config["rmax"], config["rrz"], 
                                rwnum = 0, directed = False, add_self_loop = False,
                                rand_seed = 10, 
                                feats = features, adj_matrix = adj_matrix_cython)
    
    input_data = [features_GBP, labels_full, adj_full]
    for trainer_id in trainer_id_list:
            print(f"Training >>> current tuning the hyper-param: {tune_param_name}; with a value : {tune_val} ; on trainer id: {trainer_id}" )
            # encoder = Encoder(config)
            model = RwCL_Model(config)
            
            execute_train_investigate(data_name, model, input_data, workdir, config, tune_param_name, tune_val_label, tune_val, 
                                    trainer_id = trainer_id, device = device)

            # test_encoder = Encoder(config)
            model_test = RwCL_Model(config)
            
            execute_test_investigate(data_name, model_test, input_data, workdir, config, 
                                    tune_param_name, tune_val_label, tune_val, trainer_id = trainer_id)

Total pre-computation time cost is 24.272864585000207 seconds! 
Training >>> current tuning the hyper-param: nothing; with a value : 1 ; on trainer id: 0
ckpt file already exists, so removed ...


2022-04-19 21:20:21 (INFO): Epoch   10 | total train loss: 6.407 | Trained local batch number: 4 | train time: 0.128s
2022-04-19 21:20:21 (INFO): Epoch   20 | total train loss: 6.262 | Trained local batch number: 2 | train time: 0.249s
2022-04-19 21:20:21 (INFO): Epoch   30 | total train loss: 4.988 | Trained local batch number: 6 | train time: 0.375s
2022-04-19 21:20:22 (INFO): Epoch   40 | total train loss: 6.230 | Trained local batch number: 4 | train time: 0.510s
2022-04-19 21:20:22 (INFO): Epoch   50 | total train loss: 6.190 | Trained local batch number: 2 | train time: 0.640s
2022-04-19 21:20:23 (INFO): Epoch   60 | total train loss: 4.939 | Trained local batch number: 6 | train time: 0.771s
2022-04-19 21:20:23 (INFO): Epoch   70 | total train loss: 6.178 | Trained local batch number: 4 | train time: 0.900s
2022-04-19 21:20:24 (INFO): Epoch   80 | total train loss: 6.178 | Trained local batch number: 2 | train time: 1.029s
2022-04-19 21:20:24 (INFO): Epoch   90 | total train los

/home/xiangli/projects/GCN_program/Workshop_local/EGNN_workdir_results/cora/tune_nothing/val_metric/tunelabel_1_trainer_0/val_metric.pkl, path folder already exists, so removed ...
/home/xiangli/projects/GCN_program/Workshop_local/EGNN_workdir_results/cora/tune_nothing/train_profile/tunelabel_1_trainer_0/train_profile.pkl, path folder already exists, so removed ...


2022-04-19 21:20:34 (INFO): Test metrics: | Accuracy : 0.6584194977843427 | f1_micro : 0.6584194977843427 | f1_macro : 0.6478696157691196 | NMI : 0.5086759928693642 | ARI : 0.42939509916951274 | conductance : 0.10496400151572566 | modularity : 0.7419317870327918 | test time: 0.171s


Test Runtime: 0.17s


### Post-processing

In [ ]:
for tune_val_label, tune_val in zip(tune_val_label_list, tune_val_list):
    for trainer_id in trainer_id_list:
        print(f"Validation Postprocessing >>> current tuning the hyper-param: {tune_param_name}; with a value : {tune_val} ; on trainer id: {trainer_id}" )
        Post_utils.draw_val_metrics(workdir, tune_param_name, tune_val_label, trainer_id = trainer_id)

In [ ]:
Post_utils.generate_val_all_metric(workdir, tune_param_name, tune_val_label_list, tune_val_list, trainer_id_list, real_time=False)

Post_utils.generate_val_all_metric(workdir, tune_param_name, tune_val_label_list, tune_val_list, trainer_id_list, real_time=True)

In [ ]:
Post_utils.generate_test_table(workdir, tune_param_name, 
                    tune_val_label_list, tune_val_list, trainer_id_list, skip_trainer = [])

Post_utils.plot_raw_tune_test_table(workdir, tune_param_name, 
                    tune_val_label_list, tune_val_list, trainer_id_list, skip_trainer = []) 

Post_utils.plot_stats_test_table(workdir, tune_param_name, 
                    tune_val_label_list, tune_val_list, trainer_id_list, skip_trainer = [])

### GPU flush

In [ ]:
# # free GPU memory
# !(nvidia-smi | grep 'python' | awk '{ print $5 }' | xargs -n1 kill -9 )
# !(nvidia-smi | grep 'python')